# **Worksapce set up**

In [17]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow.keras.optimizers import SGD
import numpy as np
import tempfile
import zipfile
import os

Populating the interactive namespace from numpy and matplotlib


In [19]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [20]:
# Loads the data and splits it into 60% training and 40% testing sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_test_old = y_test

x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

x_train = x_train[0:6000]
x_test = x_test[0:1000]
y_train = y_train[0:6000]
y_test = y_test[0:1000]

# **Clustering**

In [21]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/CNN_base')
base_model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test), batch_size=32)
base_model.summary()

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(base_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Epoch 1/10
188/188 [==============================] - 5s 23ms/step - loss: 0.0223 - accuracy: 0.9950 - val_loss: 0.1223 - val_accuracy: 0.9560
Epoch 2/10
188/188 [==============================] - 5s 26ms/step - loss: 0.0163 - accuracy: 0.9970 - val_loss: 0.1138 - val_accuracy: 0.9610
Epoch 3/10
188/188 [==============================] - 5s 27ms/step - loss: 0.0123 - accuracy: 0.9982 - val_loss: 0.1201 - val_accuracy: 0.9590
Epoch 4/10
188/188 [==============================] - 5s 25ms/step - loss: 0.0109 - accuracy: 0.9988 - val_loss: 0.1039 - val_accuracy: 0.9650
Epoch 5/10
188/188 [==============================] - 3s 14ms/step - loss: 0.0095 - accuracy: 0.9988 - val_loss: 0.1279 - val_accuracy: 0.9550
Epoch 6/10
188/188 [==============================] - 3s 16ms/step - loss: 0.0083 - accuracy: 0.9987 - val_loss: 0.1078 - val_accuracy: 0.9600
Epoch 7/10
188/188 [==============================] - 3s 14ms/step - loss: 0.0059 - accuracy: 0.9995 - val_loss: 0.1047 - val_accuracy: 0.9630

In [22]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

clustered_model = cluster_weights(base_model, **clustering_params)

opt = SGD(learning_rate=0.0001, momentum=0.9)
clustered_model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
clustered_model.summary()


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_conv2d_13 (ClusterW  (None, 26, 26, 32)       624       
 eights)                                                         
                                                                 
 cluster_max_pooling2d_13 (C  (None, 8, 8, 32)         0         
 lusterWeights)                                                  
                                                                 
 cluster_flatten_13 (Cluster  (None, 2048)             0         
 Weights)                                                        
                                                                 
 cluster_dense_26 (ClusterWe  (None, 30)               122926    
 ights)                                                          
                                                                 
 cluster_dense_27 (ClusterWe  (None, 10)             

In [23]:
clustered_model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test), batch_size=32)
clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(clustered_model, clustered_keras_file, include_optimizer=False)
print('Saved clustered Keras model to:', clustered_keras_file)

Epoch 1/10
188/188 [==============================] - 5s 24ms/step - loss: 0.0064 - accuracy: 0.9995 - val_loss: 0.1290 - val_accuracy: 0.9550
Epoch 2/10
188/188 [==============================] - 4s 22ms/step - loss: 0.0060 - accuracy: 0.9995 - val_loss: 0.1306 - val_accuracy: 0.9550
Epoch 3/10
188/188 [==============================] - 4s 22ms/step - loss: 0.0065 - accuracy: 0.9995 - val_loss: 0.1187 - val_accuracy: 0.9620
Epoch 4/10
188/188 [==============================] - 4s 22ms/step - loss: 0.0074 - accuracy: 0.9988 - val_loss: 0.1375 - val_accuracy: 0.9610
Epoch 5/10
188/188 [==============================] - 4s 22ms/step - loss: 0.0058 - accuracy: 0.9995 - val_loss: 0.1212 - val_accuracy: 0.9620
Epoch 6/10
188/188 [==============================] - 4s 22ms/step - loss: 0.0062 - accuracy: 0.9993 - val_loss: 0.1303 - val_accuracy: 0.9600
Epoch 7/10
188/188 [==============================] - 4s 22ms/step - loss: 0.0069 - accuracy: 0.9992 - val_loss: 0.1331 - val_accuracy: 0.9510

Saved clustered Keras model to: /tmp/tmpa6gzncj2.h5


# **Quantization aware training**

In [24]:
quantize_model = tfmot.quantization.keras.quantize_model
model = quantize_model(clustered_model)

opt = SGD(learning_rate=0.0001, momentum=0.9)
model.compile(optimizer = opt,loss = 'categorical_crossentropy' , metrics=['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (QuantizeL  (None, 28, 28, 1)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_13 (QuantizeWr  (None, 26, 26, 32)       387       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_13 (Qua  (None, 8, 8, 32)         1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_flatten_13 (QuantizeW  (None, 2048)             1         
 rapperV2)                                                       
                                                                 
 quant_dense_26 (QuantizeWra  (None, 30)             

In [25]:
# Performs quantization aware training
model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test), batch_size=32)

Epoch 1/10
188/188 [==============================] - 5s 21ms/step - loss: 0.3458 - accuracy: 0.9003 - val_loss: 0.2883 - val_accuracy: 0.9180
Epoch 2/10
188/188 [==============================] - 4s 24ms/step - loss: 0.0942 - accuracy: 0.9812 - val_loss: 0.2004 - val_accuracy: 0.9370
Epoch 3/10
188/188 [==============================] - 4s 19ms/step - loss: 0.0370 - accuracy: 0.9918 - val_loss: 0.1778 - val_accuracy: 0.9380
Epoch 4/10
188/188 [==============================] - 4s 20ms/step - loss: 0.0223 - accuracy: 0.9945 - val_loss: 0.1705 - val_accuracy: 0.9420
Epoch 5/10
188/188 [==============================] - 4s 20ms/step - loss: 0.0173 - accuracy: 0.9960 - val_loss: 0.1685 - val_accuracy: 0.9420
Epoch 6/10
188/188 [==============================] - 4s 24ms/step - loss: 0.0155 - accuracy: 0.9972 - val_loss: 0.1678 - val_accuracy: 0.9400
Epoch 7/10
188/188 [==============================] - 4s 20ms/step - loss: 0.0144 - accuracy: 0.9972 - val_loss: 0.1647 - val_accuracy: 0.9420

# **Quantization and conversion to tflite**

In [26]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('CNN_clustered.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpftcfau8r/assets


INFO:tensorflow:Assets written to: /tmp/tmpftcfau8r/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


66768

In [27]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #5 as output.

Subgraph#0 main(T#0) -> [T#15]
  Op#0 QUANTIZE(T#0) -> [T#5]
  Op#1 CONV_2D(T#5, T#6, T#2) -> [T#7]
  Op#2 MAX_POOL_2D(T#7) -> [T#8]
  Op#3 RESHAPE(T#8, T#1) -> [T#9]
  Op#4 FULLY_CONNECTED(T#9, T#10, T#3) -> [T#11]
  Op#5 FULLY_CONNECTED(T#11, T#12, T#4) -> [T#13]
  Op#6 SOFTMAX(T#13) -> [T#14]
  Op#7 DEQUANTIZE(T#14) -> [T#15]

Tensors of Subgraph#0
  T#0(serving_default_conv2d_13_input:0) shape_signature:[-1, 28, 28, 1], type:FLOAT32
  T#1(sequential_13/quant_flatten_13/Const) shape:[2], type:INT32 RO 8 bytes
  T#2(conv2d_13/bias) shape:[32], type:INT32 RO 128 bytes
  T#3(dense_26/bias) shape:[30], type:INT32 RO 120 bytes
  T#4(dense_27/bias) shape:[10], type:INT32 RO 40 bytes
  T#5(sequential_13/quantize_layer_1/AllValuesQuantize/FakeQuantWithM

In [28]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(argmax(result))

right_pred = [y_test_old[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.947


# **Conversion to C array**

In [29]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [30]:
# Write TFLite model to a C source (or header) file
with open("CNN_clustered" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "CNN_clustered"))

# **Size Comparison**

In [31]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [32]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size('CNN_clustered.tflite')))

Size of gzipped baseline Keras model: 232925.00 bytes
Size of gzipped clustered Keras model: 12584.00 bytes
Size of gzipped clustered TFlite model: 14388.00 bytes
